In [1]:
import pandas as pd
import numpy as np
import sys
import os 
#from ...util.model.training import get_percent

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
%pwd

'c:\\Users\\lowmi\\OneDrive\\Desktop\\Uni\\DSA4262\\DSA4262-ACMXZ\\preprocessing\\notebooks'

In [4]:
sys.path.append(os.path.abspath('../../util/model'))
from training import get_percent

In [5]:
%pwd

'c:\\Users\\lowmi\\OneDrive\\Desktop\\Uni\\DSA4262\\DSA4262-ACMXZ\\preprocessing\\notebooks'

In [6]:
info_data_path = "../../data/raw_data/data.info"
with open(info_data_path, 'r') as f:
    info = f.read().splitlines()
data_df1 = pd.read_parquet("../../data/merged_data/merged_data_v2_1.parquet")
data_df2 = pd.read_parquet("../../data/merged_data/merged_data_v2_2.parquet")
data_df3 = pd.read_parquet("../../data/merged_data/merged_data_v2_3.parquet")


In [7]:
print(data_df1.shape)
print(data_df2.shape)
print(data_df3.shape)


(5000000, 15)
(5000000, 15)
(1027106, 15)


In [8]:
raw_df =  pd.concat([data_df1, data_df2, data_df3])

In [9]:
raw_df.head()

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,gene_id,label
0,ENST00000000233,244,AAGACCA,185,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,185,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,185,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,185,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,185,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0


In [10]:
def model_features_and_clean(df):
    '''Function to select features for modelling and clean the initial raw data. 
        Removes rows which contain NAN values in key_columns, and converts all NAN values to 0 for non_nan_cols

    :Parameters:
    ------------
        self.df: DataFrame
            DataFrame containing raw data

    :Returns:
    ---------
        self.df: DataFrame
            DataFrame containing features required for modelling
    '''
    model_features_list =  ['transcript', 'position', 'nucleotides', 'reads_count', 'gene_id',
                            'dwellingtime_-1', 'std_-1', 'mean_-1',
                            'dwellingtime_0', 'std_0', 'mean_0',
                            'dwellingtime_+1', 'std_+1', 'mean_+1','label']
    non_nan_cols =  []
    key_columns =['label']
    df = df[model_features_list]
    df[non_nan_cols] = df[non_nan_cols].fillna(0)
    df = df.dropna(subset=key_columns)
    return df

In [11]:
cleaned_df = model_features_and_clean(raw_df)

In [12]:
cleaned_df.head()

,transcript,position,nucleotides,reads_count,gene_id,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,label
0,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,0
1,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,0
2,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,0
3,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,0
4,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,0


In [13]:
# function to clean
def feature_eng(df):
    temp = pd.DataFrame(df.groupby(['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label'], as_index=False)
                           .agg({'dwellingtime_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'dwellingtime_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'dwellingtime_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean]}))
    temp.columns = ['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label',
                        'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                        'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean',
                        'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean',
                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                        'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean',
                        'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean',
                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                        'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean',
                        'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']
    return temp

In [14]:
raw_df.columns

Index(['transcript', 'position', 'nucleotides', 'reads_count',
       'dwellingtime_-1', 'std_-1', 'mean_-1', 'dwellingtime_0', 'std_0',
       'mean_0', 'dwellingtime_+1', 'std_+1', 'mean_+1', 'gene_id', 'label'],
      dtype='object')

In [15]:
percentiles_df = feature_eng(cleaned_df)

In [19]:
percentiles_df.head()

,gene_id,transcript,position,nucleotides,reads_count,label,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENSG00000000003,ENST00000373020,1006,TAGACCT,21,0,0.007650,0.009760,0.015100,0.010726,2.8900,3.100,3.3400,3.021905,118.000,119.00,120.00,118.952381,0.005980,0.009090,0.014900,0.011677,4.3000,5.600,7.5800,6.000000,125.00,127.00,128.00,127.000000,0.003650,0.005980,0.008300,0.006490,2.0600,3.210,5.1700,3.900476,75.800,77.10,77.800,76.857143
1,ENSG00000000003,ENST00000373020,1013,AAAACTA,20,0,0.007965,0.009800,0.013175,0.011538,2.1925,2.460,2.7325,2.471500,108.000,109.00,110.00,108.900000,0.003320,0.005305,0.007763,0.006050,3.8850,4.415,4.8200,4.091500,100.00,102.00,105.00,102.675000,0.006112,0.008840,0.011600,0.010213,2.4775,2.870,4.0100,3.299000,92.125,93.25,94.975,93.765000
2,ENSG00000000003,ENST00000373020,1149,GAAACAC,22,0,0.004917,0.007155,0.009473,0.008890,2.2100,3.030,4.2975,3.440909,104.000,105.00,107.00,105.363636,0.004958,0.006885,0.010733,0.008051,1.9875,2.570,2.9875,2.652227,96.80,98.50,100.75,99.081818,0.003928,0.006205,0.007832,0.006491,1.6300,1.855,2.2675,2.028182,89.450,90.65,92.600,90.777273
3,ENSG00000000003,ENST00000373020,512,ATAACTC,20,0,0.004040,0.005990,0.008387,0.007247,1.7400,2.325,2.7050,2.359500,85.775,87.05,87.35,86.795000,0.003980,0.007735,0.013375,0.011177,2.0800,2.375,2.7300,2.496500,97.05,98.05,99.05,97.965000,0.004870,0.006870,0.011200,0.008500,1.3900,1.570,1.8750,1.588750,92.100,92.55,93.325,92.330000
4,ENSG00000000003,ENST00000373020,689,TAAACAA,21,0,0.005310,0.007640,0.014300,0.009868,1.7500,1.980,2.2300,1.972857,102.000,103.00,104.00,102.495238,0.003650,0.006420,0.008960,0.007332,1.9600,2.430,2.9100,2.551905,96.20,98.70,99.70,97.928571,0.004580,0.005730,0.008590,0.006917,1.7400,2.190,2.3700,2.198571,88.500,89.60,90.600,89.385714


In [57]:
# raw_df.loc[(raw_df['nucleotides']=='AAGACTG') & (raw_df['position'] == '2996') & (raw_df['transcript'] == 'ENST00000621914') ]

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
def split_stratified_into_train_val_test(df, random_state=None):

    # getting unique id_col and stratify_col for splitting
    id_col = ['gene_id']
    stratify_col = 'label'
    train_percent = 0.8 # percentage of train data
    validation_percent = 0.1 # percentage of validation data
    test_percent = 0.1 # percentage of test data
    position_col = ['transcript', 'position']
    target_col = 'label'


    temp_col = id_col + [stratify_col]
    df_target = df[temp_col].drop_duplicates()
    X = df_target  # Contains all columns.
    # Dataframe of just the column on which to stratify.
    y = df_target[[stratify_col]]

    # parameter checks
    if train_percent + validation_percent + test_percent != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' %
                            (train_percent, validation_percent, test_percent))

    if stratify_col not in df_target.columns:
        raise ValueError('%s is not a column in the dataframe' %
                            (stratify_col))

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, stratify=y,
                                                                    test_size=(
                                                                        1.0 - train_percent),
                                                                    random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_test_percent = test_percent / \
        (validation_percent + test_percent)
    df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp,
                                                        stratify=y_temp,
                                                        test_size=relative_test_percent,
                                                        random_state=random_state)

    assert len(df_target) == len(df_train) + \
        len(df_val) + len(df_test)

    # getting list of id_cols of each df
    list_train = np.array(df_train[id_col])
    list_train = [tuple(i) for i in list_train]
    list_test = np.array(df_test[id_col])
    list_test = [tuple(i) for i in list_test]
    list_val = np.array(df_val[id_col])
    list_val = [tuple(i) for i in list_val]

    # creating train/test/val data, removing identifying columns as they are not features.
    # identifying columns include id_col and position_col
    temp_col = id_col + position_col
    df_train = df[df[id_col].apply(
        tuple, axis=1).isin(list_train)]
    df_train.drop(columns=temp_col, inplace=True)
    df_test = df[df[id_col].apply(
        tuple, axis=1).isin(list_test)]
    df_test.drop(columns=temp_col, inplace=True)
    df_val = df[df[id_col].apply(
        tuple, axis=1).isin(list_val)]
    # df_val_id contains all features + idenifying columns
    df_val_id = df_val.copy()
    df_val.drop(columns=temp_col, inplace=True)

    # printing percentages
    print("train target percentage:", len(
        df_train[df_train[target_col] == '1'])/len(df_train))
    print("test target percentage:", len(
        df_test[df_test[target_col] == '1'])/len(df_test))
    print("val target percentage:", len(
        df_val[df_val[target_col] == '1'])/len(df_val))

    # printing df shape
    print("train data shape:", df_train.shape)
    print("validation data shape:", df_val.shape)
    print("test data shape:", df_test.shape)

    # separating df from target column: features -> X | target -> y
    X_train = df_train.drop(
        columns=target_col).reset_index(drop=True)
    y_train = pd.DataFrame(
        df_train[target_col]).reset_index(drop=True)
    X_val = df_val.drop(
        columns=target_col).reset_index(drop=True)
    y_val = pd.DataFrame(
        df_val[target_col]).reset_index(drop=True)
    X_test = df_test.drop(
        columns=target_col).reset_index(drop=True)
    y_test = pd.DataFrame(
        df_test[target_col]).reset_index(drop=True)

    return(df_train, df_test, df_val, X_train, y_train,
            X_val, y_val, X_test, y_test, df_val_id, list_train)

In [18]:
df_train, df_test, df_val, X_train, y_train, X_val, y_val, X_test, y_test, df_val_id, list_train = split_stratified_into_train_val_test(percentiles_df, random_state=42)

C:\Users\lowmi\AppData\Local\Temp\ipykernel_35136\2931462631.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(columns=temp_col, inplace=True)
C:\Users\lowmi\AppData\Local\Temp\ipykernel_35136\2931462631.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns=temp_col, inplace=True)


train target percentage: 0.049255441008018326
test target percentage: 0.06377880184331797
val target percentage: 0.07129746472410232
train data shape: (107379, 39)
validation data shape: (17434, 39)
test data shape: (16275, 39)


C:\Users\lowmi\AppData\Local\Temp\ipykernel_35136\2931462631.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val.drop(columns=temp_col, inplace=True)


In [19]:
df_train.head()

,nucleotides,reads_count,label,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,TAGACCT,21,0,0.007650,0.009760,0.015100,0.010726,2.8900,3.100,3.3400,3.021905,118.000,119.00,120.00,118.952381,0.005980,0.009090,0.014900,0.011677,4.3000,5.600,7.5800,6.000000,125.00,127.00,128.00,127.000000,0.003650,0.005980,0.008300,0.006490,2.0600,3.210,5.1700,3.900476,75.800,77.10,77.800,76.857143
1,AAAACTA,20,0,0.007965,0.009800,0.013175,0.011538,2.1925,2.460,2.7325,2.471500,108.000,109.00,110.00,108.900000,0.003320,0.005305,0.007763,0.006050,3.8850,4.415,4.8200,4.091500,100.00,102.00,105.00,102.675000,0.006112,0.008840,0.011600,0.010213,2.4775,2.870,4.0100,3.299000,92.125,93.25,94.975,93.765000
2,GAAACAC,22,0,0.004917,0.007155,0.009473,0.008890,2.2100,3.030,4.2975,3.440909,104.000,105.00,107.00,105.363636,0.004958,0.006885,0.010733,0.008051,1.9875,2.570,2.9875,2.652227,96.80,98.50,100.75,99.081818,0.003928,0.006205,0.007832,0.006491,1.6300,1.855,2.2675,2.028182,89.450,90.65,92.600,90.777273
3,ATAACTC,20,0,0.004040,0.005990,0.008387,0.007247,1.7400,2.325,2.7050,2.359500,85.775,87.05,87.35,86.795000,0.003980,0.007735,0.013375,0.011177,2.0800,2.375,2.7300,2.496500,97.05,98.05,99.05,97.965000,0.004870,0.006870,0.011200,0.008500,1.3900,1.570,1.8750,1.588750,92.100,92.55,93.325,92.330000
4,TAAACAA,21,0,0.005310,0.007640,0.014300,0.009868,1.7500,1.980,2.2300,1.972857,102.000,103.00,104.00,102.495238,0.003650,0.006420,0.008960,0.007332,1.9600,2.430,2.9100,2.551905,96.20,98.70,99.70,97.928571,0.004580,0.005730,0.008590,0.006917,1.7400,2.190,2.3700,2.198571,88.500,89.60,90.600,89.385714


In [20]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC

In [21]:
def oversample_undersample(X_train, y_train, sampling=True):
    '''Function to oversample minority and undersample majority.

    :Returns:
    ---------
        self.X_train_oversampled: DataFrame
            DataFrame contiaining oversampled and understampled training data only for first purchase date excluding target variable
    '''
    X_train_oversampled = X_train.copy()
    undersampling_strategy = 0.12
    oversampling_strategy = 0.17
    seed = 42 # random state

    if sampling:
        undersample = RandomUnderSampler(
            sampling_strategy = undersampling_strategy, random_state=seed)
        oversample = SMOTENC(categorical_features=[
                                0], sampling_strategy=oversampling_strategy, random_state=seed)
        X_train_oversampled, y_train = undersample.fit_resample(
            X_train_oversampled, y_train)
        X_train_oversampled, y_train = oversample.fit_resample(
            X_train_oversampled, y_train)

    return X_train_oversampled, y_train

In [22]:
X_train_oversampled, y_train = oversample_undersample(X_train, y_train)

In [23]:
X_train_oversampled.head()


,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,AGGACTA,117,0.007300,0.01160,0.01650,0.012815,5.6500,6.25,6.9900,6.361880,116.0,117.0,118.0,117.068376,0.006310,0.009230,0.013300,0.010990,4.8000,6.160,8.2800,6.639487,118.0,120.0,122.000,119.555556,0.004490,0.00598,0.009300,0.007264,2.4100,3.02,4.030,3.575043,88.90,89.5,90.80,89.670085
1,AGAACAG,360,0.005978,0.00899,0.01190,0.009888,5.6425,7.47,9.4400,7.757222,126.0,129.0,131.0,128.555556,0.004298,0.006310,0.009673,0.007737,2.6875,3.375,4.9525,4.138917,91.1,93.1,95.225,93.633333,0.006040,0.00842,0.011625,0.009398,1.9675,2.35,2.720,2.371108,85.70,86.6,87.60,86.599444
2,CAAACTA,64,0.003650,0.00581,0.00863,0.007196,2.0175,2.22,2.6125,2.382188,105.0,107.0,108.0,106.104687,0.004092,0.006475,0.009960,0.007692,2.4475,3.015,3.8425,3.269531,102.0,105.0,107.000,104.375000,0.007315,0.01030,0.012450,0.011004,2.3425,3.00,3.985,3.143438,92.05,93.1,93.75,93.096875
3,AGAACTT,159,0.006290,0.00823,0.01125,0.009198,5.6600,6.92,8.2850,7.082075,127.0,129.0,130.0,128.427673,0.003910,0.005980,0.009630,0.007635,2.9900,3.480,4.2900,4.013333,98.1,99.6,101.500,100.089308,0.004985,0.00720,0.009960,0.008179,1.9650,2.20,2.575,2.343711,90.75,91.8,92.60,91.806918
4,TAAACAG,27,0.003595,0.00697,0.00860,0.008041,1.7700,1.94,2.1450,1.959222,100.5,101.0,102.0,101.007407,0.004650,0.005640,0.006805,0.006277,2.4300,2.920,3.4250,3.024444,94.8,96.3,98.000,96.637037,0.005735,0.00748,0.010400,0.009347,2.3550,2.92,3.365,2.864074,84.75,85.9,87.35,86.088889


In [24]:
X_train_oversampled.shape

(51567, 38)

In [25]:
y_train['label'].value_counts()

0    44075
1     7492
Name: label, dtype: int64

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

In [33]:
X_train_oversampled.columns

Index(['nucleotides', 'reads_count', 'dwelling_time_-1_25',
       'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
       'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
       'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
       'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
       'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
       'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
       'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
       'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
       'mean_+1_50', 'mean_+1_75', 'mean_+1_mean'],
      dtype='object')

In [34]:
def encoding_train(X_train_oversampled, y_train):


    one_hot_col = ['nucleotides']
    # numeric_cols = ['dwelling_time_25', 'dwelling_time_50', 'dwelling_time_75', 'dwelling_time_mean', 
    #                         'std_25', 'std_50', 'std_75', 'std_mean', 
    #                         'mean_25', 'mean_50', 'mean_75', 'mean_mean']
    numeric_cols = ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']                            

    # piping the encoding
    numeric_encoder = Pipeline([('scale', MinMaxScaler())])
    preprocessor = ColumnTransformer(transformers=[
        ("num", numeric_encoder, numeric_cols)],
        remainder='passthrough')

    # getting list of column names to map
    columns_to_map = numeric_cols + one_hot_col

    print('columns after preprocessing :', columns_to_map,  '\n')

    # applying encoding on columns in df and creating pipeline
    X_train_enc = pd.DataFrame({col: vals for vals, col in zip(
        preprocessor.fit_transform(X_train_oversampled, y_train).T, columns_to_map)})
    pipe = Pipeline(steps=[("preprocessor", preprocessor)])
    pipe = pipe.fit(X_train_enc, y_train)

    # encoding nucleotides
    for i in range(7):
        X_train_enc['position_' + str(i) + '_A'] = 0
        X_train_enc['position_' + str(i) + '_C'] = 0
        X_train_enc['position_' + str(i) + '_G'] = 0
        X_train_enc['position_' + str(i) + '_T'] = 0
        temp = X_train_enc['nucleotides'].apply(lambda x: x[i])
        X_train_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
        X_train_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
        X_train_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
        X_train_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

    # dropping nucleotides column
    X_train_enc = X_train_enc.drop(columns=['nucleotides'])

    return X_train_enc, pipe

In [35]:
X_train_enc, pipe = encoding_train(X_train_oversampled, y_train)

columns after preprocessing : ['reads_count', 'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean', 'nucleotides'] 



C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\2671075721.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\2671075721.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\2671075721.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [36]:
X_train_enc.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,0.100207,0.412252,0.602426,0.613365,0.636974,0.520839,0.491609,0.47373,0.485616,0.670027,0.659498,0.643478,0.661934,0.33871,0.406745,0.452857,0.464628,0.406162,0.418358,0.479151,0.495439,0.65966,0.657948,0.657258,0.653869,0.28246,0.232606,0.295228,0.295901,0.159468,0.156971,0.202229,0.208612,0.694184,0.630088,0.399558,0.528776,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
1,0.35124,0.302773,0.42655,0.393795,0.434309,0.519989,0.612043,0.684755,0.621482,0.853346,0.874552,0.869565,0.871885,0.167869,0.205781,0.268228,0.254327,0.169468,0.172551,0.235735,0.251503,0.121121,0.1167,0.11744,0.121675,0.484217,0.433512,0.424251,0.465269,0.10601,0.095106,0.09793,0.094146,0.574109,0.527434,0.328918,0.435668,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
2,0.045455,0.110099,0.212264,0.237709,0.247886,0.108874,0.093781,0.096684,0.09811,0.468378,0.480287,0.469565,0.461551,0.150467,0.217137,0.28286,0.251394,0.142577,0.140777,0.154535,0.166693,0.339339,0.356137,0.354839,0.342206,0.650179,0.588308,0.470033,0.592796,0.151314,0.155125,0.198646,0.167576,0.812383,0.757522,0.46468,0.632683,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
3,0.143595,0.328642,0.375337,0.362768,0.386541,0.521973,0.557749,0.585271,0.555742,0.871677,0.874552,0.852174,0.869547,0.134975,0.18307,0.266064,0.247711,0.203361,0.181818,0.187271,0.239252,0.261261,0.247485,0.243952,0.254219,0.346892,0.333059,0.331853,0.368514,0.105708,0.081256,0.086385,0.091541,0.763602,0.711504,0.439294,0.593569,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
4,0.007231,0.105546,0.290431,0.236277,0.306388,0.080805,0.06614,0.056417,0.056926,0.385885,0.37276,0.365217,0.368388,0.197793,0.15967,0.12228,0.159962,0.140616,0.132392,0.123994,0.142784,0.195195,0.181087,0.173387,0.183342,0.444517,0.356114,0.356271,0.461286,0.152824,0.147738,0.149283,0.141015,0.538462,0.502655,0.3234,0.420187,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0


In [37]:
def encoding_test_val(X_test, X_val, test=True):

    one_hot_col = ['nucleotides']

    numeric_cols = ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']   
    columns_to_map = numeric_cols + one_hot_col

    # applying encoding on columns in df
    if test == True:  # for test
        X_test_enc = pd.DataFrame({col: vals for vals, col in zip(
            pipe.transform(X_test).T, columns_to_map)})
        # encoding nucleotides
        for i in range(7):
            X_test_enc['position_' + str(i) + '_A'] = 0
            X_test_enc['position_' + str(i) + '_C'] = 0
            X_test_enc['position_' + str(i) + '_G'] = 0
            X_test_enc['position_' + str(i) + '_T'] = 0
            temp = X_test_enc['nucleotides'].apply(lambda x: x[i])
            X_test_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
            X_test_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
            X_test_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
            X_test_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

        # dropping nucleotides column
        X_test_enc = X_test_enc.drop(columns=['nucleotides'])

        return X_test_enc
    else:  # for validation
        X_val_enc = pd.DataFrame({col: vals for vals, col in zip(
            pipe.transform(X_val).T, columns_to_map)})
        # encoding nucleotides
        for i in range(5):
            X_val_enc['position_' + str(i) + '_A'] = 0
            X_val_enc['position_' + str(i) + '_C'] = 0
            X_val_enc['position_' + str(i) + '_G'] = 0
            X_val_enc['position_' + str(i) + '_T'] = 0
            temp = X_val_enc['nucleotides'].apply(lambda x: x[i])
            X_val_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
            X_val_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
            X_val_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
            X_val_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

        # dropping nucleotides column
        X_val_enc = X_val_enc.drop(columns=['nucleotides'])

        return X_val_enc

In [38]:
X_test_enc = encoding_test_val(X_test, X_val)

C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\3423641716.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\3423641716.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\3423641716.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [39]:
X_test_enc.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,28.0,0.006128,0.007705,0.012475,0.009604,2.5225,3.01,3.71,3.236786,118.0,119.0,119.25,118.821429,0.004247,0.00697,0.00921,0.007826,5.1525,7.68,8.79,7.077857,119.0,123.0,125.0,122.285714,0.004635,0.006395,0.00802,0.00628,2.2575,2.63,3.145,2.951786,86.875,87.6,88.1,87.796429,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
1,25.0,0.00498,0.00598,0.00996,0.007896,2.6,3.5,4.19,3.886,118.0,119.0,122.0,119.76,0.00494,0.00697,0.00915,0.007654,3.71,4.61,4.97,4.662,124.0,125.0,126.0,125.08,0.00398,0.00472,0.00564,0.005261,2.11,2.86,4.1,3.3256,84.6,86.7,87.9,86.4,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,30.0,0.004685,0.007245,0.010228,0.008172,4.3725,5.685,6.8,5.361333,126.25,130.0,131.0,127.466667,0.00483,0.006655,0.00947,0.007708,3.0275,3.375,3.7925,3.402,102.25,104.0,106.0,104.146667,0.004537,0.00664,0.008217,0.006792,3.1225,3.555,4.185,3.603667,92.7,94.4,96.675,94.09,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
3,29.0,0.00466,0.00593,0.00664,0.00608,3.36,3.86,4.47,4.323793,99.4,102.0,104.0,101.303448,0.00469,0.0073,0.0106,0.00812,6.67,8.43,10.0,8.573448,116.0,118.0,120.0,117.551724,0.00398,0.00531,0.0073,0.006557,2.05,2.88,4.09,3.34,76.1,77.2,79.4,77.1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1
4,29.0,0.00398,0.00697,0.0103,0.008198,2.11,3.37,4.34,3.382069,106.0,108.0,110.0,107.758621,0.00498,0.00977,0.013,0.009567,3.37,3.84,4.2,3.775172,103.0,105.0,107.0,104.965517,0.00447,0.00631,0.00805,0.007314,2.05,2.51,2.95,2.751724,91.5,92.3,93.0,92.37931,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0


In [40]:
X_val_enc = encoding_test_val(X_test, X_val, test = False)

C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\3423641716.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\3423641716.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_24672\3423641716.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [41]:
X_val_enc.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T
0,39.0,0.00415,0.00664,0.0108,0.00785,3.96,4.57,5.19,4.744103,116.0,118.0,120.5,117.589744,0.00398,0.00686,0.0093,0.007363,4.475,6.0,9.135,6.792308,124.5,127.0,129.0,126.923077,0.00365,0.00564,0.008465,0.006415,1.93,4.39,5.825,4.245641,77.6,78.5,80.05,78.797436,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
1,32.0,0.007633,0.00986,0.014775,0.012129,7.1275,8.425,10.15,8.571563,127.5,130.0,132.0,129.53125,0.006533,0.00772,0.009113,0.008446,2.44,3.19,3.8225,3.315625,95.2,96.0,98.3,96.425,0.004478,0.00581,0.008712,0.006396,1.93,2.225,2.8175,2.381875,88.0,89.05,90.6,88.965625,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
2,31.0,0.00398,0.00564,0.00863,0.00689,2.715,3.16,4.33,3.95871,121.0,125.0,126.5,124.193548,0.00601,0.00896,0.01415,0.010207,5.48,6.74,7.75,6.638387,128.0,130.0,131.0,129.290323,0.00432,0.00598,0.00892,0.007296,2.8,4.26,6.48,4.819677,79.6,81.4,82.55,81.148387,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0
3,34.0,0.005088,0.007625,0.01074,0.008839,5.895,7.67,9.4875,7.826765,132.0,133.5,135.0,133.411765,0.003832,0.00414,0.00593,0.005436,2.825,3.7,4.39,3.857647,95.325,96.9,99.225,97.452941,0.004537,0.005475,0.007508,0.006321,2.2,2.595,2.905,2.625294,83.025,85.2,87.8,85.794118,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0
4,30.0,0.00465,0.00546,0.007788,0.006908,5.7775,8.59,9.6475,7.989333,115.0,118.5,122.0,118.833333,0.006437,0.008485,0.011975,0.009932,2.5,2.82,3.2725,2.961667,95.45,96.3,96.95,96.31,0.004732,0.00571,0.00797,0.00776,2.525,2.935,3.4375,3.026667,87.5,88.95,90.225,88.57,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0


In [45]:
X_train_25 = X_train_enc.drop(columns=['dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean',
                                        'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_50', 'mean_0_75', 'mean_0_mean',
                                        'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean'])

X_train_50 = X_train_enc.drop(columns=['dwelling_time_-1_25', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_75', 'std_-1_mean', 'mean_-1_25', 'mean_-1_75', 'mean_-1_mean',
                                        'dwelling_time_0_25', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_25', 'std_0_75', 'std_0_mean', 'mean_0_25', 'mean_0_75', 'mean_0_mean',
                                        'dwelling_time_+1_25', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_75', 'std_+1_mean', 'mean_+1_25', 'mean_+1_75', 'mean_+1_mean'])

X_train_75 = X_train_enc.drop(columns=['dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_50', 'std_-1_mean', 'mean_-1_25', 'mean_-1_50', 'mean_-1_mean',
                                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_mean', 'std_0_25', 'std_0_50', 'std_0_mean', 'mean_0_25', 'mean_0_50', 'mean_0_mean',
                                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_50', 'std_+1_mean', 'mean_+1_25', 'mean_+1_50', 'mean_+1_mean'])

X_train_mean = X_train_enc.drop(columns=['dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'std_-1_25', 'std_-1_50', 'std_-1_75', 'mean_-1_25', 'mean_-1_50', 'mean_-1_75',
                                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'std_0_25', 'std_0_50', 'std_0_75', 'mean_0_25', 'mean_0_50', 'mean_0_75',
                                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'std_+1_25', 'std_+1_50', 'std_+1_75', 'mean_+1_25', 'mean_+1_50', 'mean_+1_75'])

In [44]:
X_train_25.columns

Index(['reads_count', 'dwelling_time_-1_25', 'std_-1_25', 'mean_-1_25',
       'dwelling_time_0_25', 'std_0_25', 'mean_0_25', 'dwelling_time_+1_25',
       'std_+1_25', 'mean_+1_25', 'position_0_A', 'position_0_C',
       'position_0_G', 'position_0_T', 'position_1_A', 'position_1_C',
       'position_1_G', 'position_1_T', 'position_2_A', 'position_2_C',
       'position_2_G', 'position_2_T', 'position_3_A', 'position_3_C',
       'position_3_G', 'position_3_T', 'position_4_A', 'position_4_C',
       'position_4_G', 'position_4_T', 'position_5_A', 'position_5_C',
       'position_5_G', 'position_5_T', 'position_6_A', 'position_6_C',
       'position_6_G', 'position_6_T'],
      dtype='object')

In [46]:
X_train_50.columns

Index(['reads_count', 'dwelling_time_-1_50', 'std_-1_50', 'mean_-1_50',
       'dwelling_time_0_50', 'std_0_50', 'mean_0_50', 'dwelling_time_+1_50',
       'std_+1_50', 'mean_+1_50', 'position_0_A', 'position_0_C',
       'position_0_G', 'position_0_T', 'position_1_A', 'position_1_C',
       'position_1_G', 'position_1_T', 'position_2_A', 'position_2_C',
       'position_2_G', 'position_2_T', 'position_3_A', 'position_3_C',
       'position_3_G', 'position_3_T', 'position_4_A', 'position_4_C',
       'position_4_G', 'position_4_T', 'position_5_A', 'position_5_C',
       'position_5_G', 'position_5_T', 'position_6_A', 'position_6_C',
       'position_6_G', 'position_6_T'],
      dtype='object')

In [47]:
X_train_75.columns

Index(['reads_count', 'dwelling_time_-1_75', 'std_-1_75', 'mean_-1_75',
       'dwelling_time_0_75', 'std_0_75', 'mean_0_75', 'dwelling_time_+1_75',
       'std_+1_75', 'mean_+1_75', 'position_0_A', 'position_0_C',
       'position_0_G', 'position_0_T', 'position_1_A', 'position_1_C',
       'position_1_G', 'position_1_T', 'position_2_A', 'position_2_C',
       'position_2_G', 'position_2_T', 'position_3_A', 'position_3_C',
       'position_3_G', 'position_3_T', 'position_4_A', 'position_4_C',
       'position_4_G', 'position_4_T', 'position_5_A', 'position_5_C',
       'position_5_G', 'position_5_T', 'position_6_A', 'position_6_C',
       'position_6_G', 'position_6_T'],
      dtype='object')

In [48]:
X_train_mean.columns

Index(['reads_count', 'dwelling_time_-1_mean', 'std_-1_mean', 'mean_-1_mean',
       'dwelling_time_0_mean', 'std_0_mean', 'mean_0_mean',
       'dwelling_time_+1_mean', 'std_+1_mean', 'mean_+1_mean', 'position_0_A',
       'position_0_C', 'position_0_G', 'position_0_T', 'position_1_A',
       'position_1_C', 'position_1_G', 'position_1_T', 'position_2_A',
       'position_2_C', 'position_2_G', 'position_2_T', 'position_3_A',
       'position_3_C', 'position_3_G', 'position_3_T', 'position_4_A',
       'position_4_C', 'position_4_G', 'position_4_T', 'position_5_A',
       'position_5_C', 'position_5_G', 'position_5_T', 'position_6_A',
       'position_6_C', 'position_6_G', 'position_6_T'],
      dtype='object')

In [50]:
X_train_mean.head()

,reads_count,dwelling_time_-1_mean,std_-1_mean,mean_-1_mean,dwelling_time_0_mean,std_0_mean,mean_0_mean,dwelling_time_+1_mean,std_+1_mean,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,0.100207,0.636974,0.485616,0.661934,0.464628,0.495439,0.653869,0.295901,0.208612,0.528776,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
1,0.35124,0.434309,0.621482,0.871885,0.254327,0.251503,0.121675,0.465269,0.094146,0.435668,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
2,0.045455,0.247886,0.09811,0.461551,0.251394,0.166693,0.342206,0.592796,0.167576,0.632683,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
3,0.143595,0.386541,0.555742,0.869547,0.247711,0.239252,0.254219,0.368514,0.091541,0.593569,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
4,0.007231,0.306388,0.056926,0.368388,0.159962,0.142784,0.183342,0.461286,0.141015,0.420187,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0


In [56]:
%cd ../../data/new_preprocessed_data

c:\Users\Zoe\DSA4262-ACMXZ\data\new_preprocessed_data


In [58]:
%pwd

'c:\\Users\\Zoe\\DSA4262-ACMXZ\\data\\new_preprocessed_data'

'c:\\Users\\Zoe\\DSA4262-ACMXZ\\preprocessing\\notebooks'

In [61]:
#into archive
percentiles_df.to_parquet("archive/df_all_features.parquet") # df containing all features, before splitting/encoding
df_test.to_parquet("archive/df_test.parquet") # df test containing all features and target, after splitting, before encoding
df_train.to_parquet("archive/df_train.parquet") # df trin containing all features and target, after splitting, before encoding
df_val_id.to_parquet("archive/df_val_id.parquet") # df val containing all features, identity cols and target, after splitting, before encoding
df_val.to_parquet("archive/df_val.parquet") # df val containing all features and target, after splitting, before encoding
X_test.to_parquet("archive/X_test.parquet") # df test containing all features, after splitting, before encoding
X_train.to_parquet("archive/X_train.parquet") # df train containing all features, after splitting, before encoding
X_val.to_parquet("archive/X_val.parquet") # df val containing all features, after splitting, before encoding

#into test
X_test_enc.to_parquet("test/X_test_enc.parquet") # df test containing all features, after encoding and oversampling
y_test.to_parquet("test/y_test.parquet") # target test, after splitting, before encoding

#into training
X_train_25.to_parquet("training/X_train_25.parquet")  # final df containing 25 percentile features for modelling
X_train_50.to_parquet("training/X_train_50.parquet") # final df containing 50 percentile features for modelling
X_train_75.to_parquet("training/X_train_75.parquet") # final df containing 75 percentile features for modelling
X_train_enc.to_parquet("training/X_train_enc.parquet") # df train containing all features, after encoding and oversampling
X_train_mean.to_parquet("training/X_train_mean.parquet") # final df containing mean features for modelling
y_train.to_parquet("training/y_train.parquet") # target train, after splitting, before encoding

#into validation
X_val_enc.to_parquet("validation/X_val_enc.parquet") # df val containing all features, after encoding and oversampling
y_val.to_parquet("validation/y_val.parquet") # target val, after splitting, before encoding
